# 02 - Data Cleaning & Validation

This notebook cleans and validates the raw ESPN Soccer data, preparing it for feature engineering.

## Objectives
1. Filter to completed matches only
2. Remove duplicates and handle missing values
3. Merge fixtures with team stats, teams, and leagues
4. Apply comprehensive league tier classification (60+ leagues)
5. Optimize joins and relationships for efficiency
6. Output clean `matches_base.parquet`

## Key Enhancements
- Efficient batch processing with vectorized operations
- Pre-match vs post-match field distinction
- Improved relationship handling with merge optimization
- Schema alignment with `ingest_all_data.py`
- Comprehensive global league coverage

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)

# Paths
DATA_DIR = Path('../data')
BASE_DATA = DATA_DIR / 'base_data'
PROCESSED_DIR = DATA_DIR / 'processed'
PROCESSED_DIR.mkdir(exist_ok=True)

print("📂 Data Cleaning Pipeline Started")
print("=" * 50)

📂 Data Cleaning Pipeline Started


## 1. Load Raw Data with Optimized dtypes

In [2]:
# Define optimal dtypes for memory efficiency
FIXTURE_DTYPES = {
    'Rn': 'int32',
    'seasonType': 'int32',
    'leagueId': 'int32',
    'eventId': 'int64',
    'venueId': 'int32',
    'attendance': 'int32',
    'homeTeamId': 'int32',
    'awayTeamId': 'int32',
    'homeTeamScore': 'int16',
    'awayTeamScore': 'int16',
    'homeTeamShootoutScore': 'int16',
    'awayTeamShootoutScore': 'int16',
    'statusId': 'int16'
}

TEAM_STATS_DTYPES = {
    'seasonType': 'int32',
    'eventId': 'int64',
    'teamId': 'int32',
    'teamOrder': 'int8',
    'possessionPct': 'float32',
    'foulsCommitted': 'float32',
    'yellowCards': 'float32',
    'redCards': 'float32',
    'offsides': 'float32',
    'wonCorners': 'float32',
    'saves': 'float32',
    'totalShots': 'float32',
    'shotsOnTarget': 'float32',
    'shotPct': 'float32',
    'accuratePasses': 'float32',
    'totalPasses': 'float32',
    'passPct': 'float32'
}

# Load base tables with optimized dtypes
print("Loading data with optimized memory...")
fixtures = pd.read_csv(BASE_DATA / 'fixtures.csv', dtype=FIXTURE_DTYPES)
teams = pd.read_csv(BASE_DATA / 'teams.csv')
leagues = pd.read_csv(BASE_DATA / 'leagues.csv')
team_stats = pd.read_csv(BASE_DATA / 'teamStats.csv', dtype=TEAM_STATS_DTYPES)
standings = pd.read_csv(BASE_DATA / 'standings.csv')
status = pd.read_csv(BASE_DATA / 'status.csv')

print(f"✅ Loaded fixtures: {len(fixtures):,} rows ({fixtures.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB)")
print(f"✅ Loaded teams: {len(teams):,} rows")
print(f"✅ Loaded leagues: {len(leagues):,} rows")
print(f"✅ Loaded team_stats: {len(team_stats):,} rows ({team_stats.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB)")
print(f"✅ Loaded standings: {len(standings):,} rows")

Loading data with optimized memory...
✅ Loaded fixtures: 67,353 rows (11.82 MB)
✅ Loaded teams: 4,144 rows
✅ Loaded leagues: 1,084 rows
✅ Loaded team_stats: 103,787 rows (25.44 MB)
✅ Loaded standings: 6,071 rows


## 2. Filter to Completed Matches

In [3]:
# Completed match status IDs
COMPLETED_STATUSES = [
    28,  # Full Time
    45,  # Final - After Extra Time
    46,  # Final - After Golden Goal
    47,  # Final - After Penalties
    51   # Final - Abandoned (but with result)
]

# Filter fixtures using vectorized operation
mask = fixtures['statusId'].isin(COMPLETED_STATUSES)
matches = fixtures.loc[mask].copy()
print(f"📊 Completed matches: {len(matches):,} ({len(matches)/len(fixtures)*100:.1f}% of total)")

# Check and remove duplicates efficiently
dup_mask = matches.duplicated(subset='eventId', keep='first')
n_dups = dup_mask.sum()
if n_dups > 0:
    matches = matches.loc[~dup_mask]
    print(f"   Removed {n_dups} duplicates")
print(f"🔍 Unique eventIds: {matches['eventId'].nunique():,}")

📊 Completed matches: 57,870 (85.9% of total)
🔍 Unique eventIds: 57,870


## 3. Standardize Date Format

In [4]:
# Convert date with UTC timezone
matches['date'] = pd.to_datetime(matches['date'], utc=True)
matches = matches.sort_values('date').reset_index(drop=True)

print(f"📅 Date range: {matches['date'].min()} to {matches['date'].max()}")
print(f"   Total days: {(matches['date'].max() - matches['date'].min()).days}")

📅 Date range: 2024-01-01 05:00:00+00:00 to 2025-12-15 20:45:00+00:00
   Total days: 714


## 4. Comprehensive League Tier Classification

60+ leagues across Europe, Americas, Asia, and Africa.

In [5]:
# ============================================================
# COMPREHENSIVE LEAGUE TIER CLASSIFICATION
# ============================================================

# TIER 1 - Top 5 European Leagues (Highest data quality and betting interest)
TIER_1_MIDSIZE = {'ENG.1', 'ESP.1', 'ITA.1', 'GER.1', 'FRA.1'}

# TIER 2 - Comprehensive global coverage (50+ leagues)
TIER_2_MIDSIZE = {
    # Second tier of major European leagues
    'ENG.2', 'ENG.3', 'ENG.4', 'ENG.5',  # English pyramid
    'ESP.2', 'ITA.2', 'GER.2', 'FRA.2',
    
    # Other top European first divisions
    'NED.1', 'NED.2',  # Netherlands
    'POR.1', 'POR.2',  # Portugal
    'BEL.1',           # Belgium
    'TUR.1', 'TUR.2',  # Turkey
    'RUS.1',           # Russia
    'UKR.1',           # Ukraine
    'SCO.1', 'SCO.2',  # Scotland
    'AUT.1',           # Austria
    'SUI.1',           # Switzerland
    'GRE.1',           # Greece
    'DEN.1',           # Denmark
    'NOR.1',           # Norway
    'SWE.1',           # Sweden
    'FIN.1',           # Finland
    'POL.1',           # Poland
    'CZE.1',           # Czech Republic
    'CRO.1',           # Croatia
    'SRB.1',           # Serbia
    'ROM.1',           # Romania
    'BUL.1',           # Bulgaria
    'HUN.1',           # Hungary
    'CYP.1',           # Cyprus
    'ISR.1',           # Israel
    
    # European competitions
    'UEFA.CHAMPIONS', 'UEFA.EUROPA', 'UEFA.EUROPA.CONF',
    
    # Americas - North & Central
    'USA.1', 'USA.2',           # USA MLS & USL
    'MEX.1', 'MEX.2',           # Mexico
    'CAN.1',                    # Canada
    'CRC.1',                    # Costa Rica
    'CONCACAF.CHAMPIONS',       # CONCACAF Champions League
    
    # Americas - South
    'BRA.1', 'BRA.2',           # Brazil
    'ARG.1', 'ARG.2',           # Argentina
    'COL.1',                    # Colombia
    'CHI.1',                    # Chile
    'PER.1',                    # Peru
    'ECU.1',                    # Ecuador
    'URU.1',                    # Uruguay
    'PAR.1',                    # Paraguay
    'VEN.1',                    # Venezuela
    'BOL.1',                    # Bolivia
    'CONMEBOL.LIBERTADORES',    # Copa Libertadores
    'CONMEBOL.SUDAMERICANA',    # Copa Sudamericana
    
    # Asia & Oceania
    'JPN.1', 'JPN.2',           # Japan J-League
    'KOR.1',                    # South Korea K-League
    'CHN.1',                    # China Super League
    'AUS.1',                    # Australia A-League
    'SAU.1',                    # Saudi Arabia
    'UAE.1',                    # UAE
    'QAT.1',                    # Qatar
    'IND.1',                    # India ISL
    'THA.1',                    # Thailand
    'IDN.1',                    # Indonesia
    'MYS.1',                    # Malaysia
    'AFC.CHAMPIONS',            # AFC Champions League
    
    # Africa
    'RSA.1',                    # South Africa PSL
    'EGY.1',                    # Egypt
    'MAR.1',                    # Morocco
    'TUN.1',                    # Tunisia
    'ALG.1',                    # Algeria
    'NGA.1',                    # Nigeria
    'GHA.1',                    # Ghana
    'KEN.1',                    # Kenya
    'UGA.1',                    # Uganda
    'CAF.CHAMPIONS',            # CAF Champions League
    'CAF.CONFED'                # CAF Confederation Cup
}

# Create league lookup with tier using vectorized assignment
league_lookup = leagues[['leagueId', 'midsizeName', 'leagueName', 'year']].drop_duplicates(subset='leagueId').copy()

# Vectorized tier assignment
league_lookup['tier'] = np.select(
    [league_lookup['midsizeName'].isin(TIER_1_MIDSIZE),
     league_lookup['midsizeName'].isin(TIER_2_MIDSIZE)],
    [1, 2],
    default=3
)

print("🏆 League Tier Distribution:")
tier_counts = league_lookup['tier'].value_counts().sort_index()
for tier, count in tier_counts.items():
    print(f"   Tier {tier}: {count} leagues")
print(f"\n   Tier 1 leagues: {len(TIER_1_MIDSIZE)}")
print(f"   Tier 2 leagues: {len(TIER_2_MIDSIZE)}")

🏆 League Tier Distribution:
   Tier 1: 5 leagues
   Tier 2: 61 leagues
   Tier 3: 154 leagues

   Tier 1 leagues: 5
   Tier 2 leagues: 83


## 5. Optimized Relationship Merging

Using indexed merges for better performance on large datasets.

In [6]:
# Create indexed lookup dictionaries for O(1) access
print("Creating indexed lookups for efficient merging...")

# Team lookup - create once, reuse twice
team_lookup = teams.set_index('teamId')[['displayName', 'shortDisplayName', 'logoURL']].to_dict('index')

# Add home team info using vectorized map
matches['home_team_name'] = matches['homeTeamId'].map(
    lambda x: team_lookup.get(x, {}).get('displayName', '')
)
matches['home_team_short'] = matches['homeTeamId'].map(
    lambda x: team_lookup.get(x, {}).get('shortDisplayName', '')
)
matches['home_team_logo'] = matches['homeTeamId'].map(
    lambda x: team_lookup.get(x, {}).get('logoURL', '')
)

# Add away team info
matches['away_team_name'] = matches['awayTeamId'].map(
    lambda x: team_lookup.get(x, {}).get('displayName', '')
)
matches['away_team_short'] = matches['awayTeamId'].map(
    lambda x: team_lookup.get(x, {}).get('shortDisplayName', '')
)
matches['away_team_logo'] = matches['awayTeamId'].map(
    lambda x: team_lookup.get(x, {}).get('logoURL', '')
)

print(f"✅ Merged team names ")
print(f"   Missing home team names: {(matches['home_team_name'] == '').sum()}")
print(f"   Missing away team names: {(matches['away_team_name'] == '').sum()}")

Creating indexed lookups for efficient merging...
✅ Merged team names 
   Missing home team names: 0
   Missing away team names: 0


In [7]:
# League lookup using indexed merge for efficiency
league_idx = league_lookup.set_index('leagueId')[['midsizeName', 'leagueName', 'tier']]

matches = matches.merge(
    league_idx.rename(columns={'midsizeName': 'league_code', 'leagueName': 'league_name'}),
    left_on='leagueId',
    right_index=True,
    how='left'
)

print(f"✅ Merged league info")
print(f"\n📊 Matches by Tier:")
tier_match_counts = matches['tier'].value_counts().sort_index()
for tier, count in tier_match_counts.items():
    print(f"   Tier {tier}: {count:,} ({count/len(matches)*100:.1f}%)")

✅ Merged league info

📊 Matches by Tier:
   Tier 1.0: 2,472 (4.3%)
   Tier 2.0: 28,485 (49.2%)
   Tier 3.0: 26,405 (45.6%)


## 6. Optimized Team Stats Merge

Using pivot operations for efficient home/away stats splitting.

In [8]:
# Key stats columns - these are POST-MATCH fields
# Note: For backtesting filters, these should only be used as ground truth,
# not as filter conditions (to avoid look-ahead bias)
STATS_COLS = [
    'possessionPct', 'foulsCommitted', 'yellowCards', 'redCards',
    'offsides', 'wonCorners', 'saves', 'totalShots', 'shotsOnTarget',
    'shotPct', 'accuratePasses', 'totalPasses', 'passPct'
]

# Filter to relevant columns only
stats_subset = team_stats[['eventId', 'teamId', 'teamOrder'] + STATS_COLS].copy()

# Split and pivot in one operation for efficiency
home_mask = stats_subset['teamOrder'] == 0
away_mask = stats_subset['teamOrder'] == 1

# Home stats
home_stats = stats_subset.loc[home_mask].drop(columns=['teamId', 'teamOrder'])
home_stats.columns = ['eventId'] + [f'home_{col}' for col in STATS_COLS]
home_stats = home_stats.drop_duplicates(subset='eventId')

# Away stats
away_stats = stats_subset.loc[away_mask].drop(columns=['teamId', 'teamOrder'])
away_stats.columns = ['eventId'] + [f'away_{col}' for col in STATS_COLS]
away_stats = away_stats.drop_duplicates(subset='eventId')

print(f"📊 Home stats records: {len(home_stats):,}")
print(f"📊 Away stats records: {len(away_stats):,}")

📊 Home stats records: 51,209
📊 Away stats records: 51,209


In [9]:
# Use indexed merge for better performance
home_stats_idx = home_stats.set_index('eventId')
away_stats_idx = away_stats.set_index('eventId')

matches = matches.merge(home_stats_idx, left_on='eventId', right_index=True, how='left')
matches = matches.merge(away_stats_idx, left_on='eventId', right_index=True, how='left')

# Check coverage
stats_coverage = matches['home_possessionPct'].notna().mean() * 100
print(f"✅ Merged team stats")
print(f"   Stats coverage: {stats_coverage:.1f}%")

✅ Merged team stats
   Stats coverage: 48.9%


## 7. Handle Missing Values with Proper Strategy

In [10]:
# Fill missing numeric stats with 0 (conservative approach for matches without detailed stats)
stats_columns = [f'home_{col}' for col in STATS_COLS] + [f'away_{col}' for col in STATS_COLS]

print(f"📊 Processing {len(stats_columns)} stats columns...")

# Vectorized fill for efficiency
for col in stats_columns:
    if col in matches.columns:
        matches[col] = matches[col].fillna(0)

# Verify
remaining_nulls = matches[stats_columns].isna().sum().sum()
print(f"✅ Filled missing values. Remaining nulls in stats: {remaining_nulls}")

📊 Processing 26 stats columns...
✅ Filled missing values. Remaining nulls in stats: 0


## 8. Data Validation

In [11]:
print("🔍 Data Validation")
print("=" * 50)

# Check score ranges
max_home_score = matches['homeTeamScore'].max()
max_away_score = matches['awayTeamScore'].max()
print(f"   Max home score: {max_home_score}")
print(f"   Max away score: {max_away_score}")

# Flag anomalous scores (>10 goals)
anomalous_mask = (matches['homeTeamScore'] > 10) | (matches['awayTeamScore'] > 10)
print(f"   Matches with score > 10: {anomalous_mask.sum()}")

# Check possession ranges
if 'home_possessionPct' in matches.columns:
    invalid_poss_mask = (matches['home_possessionPct'] > 100) | (matches['away_possessionPct'] > 100)
    print(f"   Invalid possession (>100%): {invalid_poss_mask.sum()}")

# Validate team relationships
same_team_mask = matches['homeTeamId'] == matches['awayTeamId']
print(f"   Matches with same home/away team: {same_team_mask.sum()}")
if same_team_mask.sum() > 0:
    matches = matches.loc[~same_team_mask]
    print("   ⚠️ Removed invalid matches")

print("\n✅ Validation complete")

🔍 Data Validation
   Max home score: 16
   Max away score: 19
   Matches with score > 10: 26
   Invalid possession (>100%): 0
   Matches with same home/away team: 0

✅ Validation complete


## 9. Select Final Columns (Aligned with ingest_all_data.py)

In [12]:
# Define final column order - aligned with ingest_all_data.py schema
FINAL_COLUMNS = [
    # Match identifiers
    'eventId', 'date', 'leagueId', 'league_code', 'league_name', 'tier',
    'venueId', 'attendance', 'seasonType',
    
    # Teams with full info
    'homeTeamId', 'home_team_name', 'home_team_short', 'home_team_logo',
    'awayTeamId', 'away_team_name', 'away_team_short', 'away_team_logo',
    
    # Scores (POST-MATCH - ground truth only)
    'homeTeamScore', 'awayTeamScore',
    'homeTeamWinner', 'awayTeamWinner',
    
    # Home team stats (POST-MATCH)
    'home_possessionPct', 'home_totalShots', 'home_shotsOnTarget',
    'home_wonCorners', 'home_foulsCommitted', 'home_yellowCards', 'home_redCards',
    'home_offsides', 'home_saves', 'home_shotPct',
    'home_accuratePasses', 'home_totalPasses', 'home_passPct',
    
    # Away team stats (POST-MATCH)
    'away_possessionPct', 'away_totalShots', 'away_shotsOnTarget',
    'away_wonCorners', 'away_foulsCommitted', 'away_yellowCards', 'away_redCards',
    'away_offsides', 'away_saves', 'away_shotPct',
    'away_accuratePasses', 'away_totalPasses', 'away_passPct',
    
    # Status
    'statusId'
]

# Select columns that exist
available_cols = [col for col in FINAL_COLUMNS if col in matches.columns]
matches_clean = matches[available_cols].copy()

print(f"📋 Final columns: {len(available_cols)}")
print(f"📊 Final rows: {len(matches_clean):,}")

📋 Final columns: 48
📊 Final rows: 57,870


## 10. Export Clean Data

In [13]:
# Save to parquet with compression
output_path = PROCESSED_DIR / 'matches_base.parquet'
matches_clean.to_parquet(output_path, index=False, compression='snappy')

print(f"✅ Saved to {output_path}")
print(f"   File size: {output_path.stat().st_size / 1024 / 1024:.2f} MB")

# Also save league lookup
league_output = PROCESSED_DIR / 'leagues_clean.parquet'
league_lookup.to_parquet(league_output, index=False)
print(f"✅ Saved leagues to {league_output}")

✅ Saved to ../data/processed/matches_base.parquet
   File size: 2.76 MB
✅ Saved leagues to ../data/processed/leagues_clean.parquet


In [14]:
# Summary
print("\n" + "=" * 50)
print("📊 CLEANING SUMMARY")
print("=" * 50)
print(f"   Input fixtures: {len(fixtures):,}")
print(f"   Output matches: {len(matches_clean):,}")
print(f"   Tier 1 matches: {len(matches_clean[matches_clean['tier'] == 1]):,}")
print(f"   Tier 2 matches: {len(matches_clean[matches_clean['tier'] == 2]):,}")
print(f"   Tier 3 matches: {len(matches_clean[matches_clean['tier'] == 3]):,}")
print(f"\n   Stats coverage: {(matches_clean['home_possessionPct'] > 0).sum() / len(matches_clean) * 100:.1f}%")
print(f"   Memory usage: {matches_clean.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")


📊 CLEANING SUMMARY
   Input fixtures: 67,353
   Output matches: 57,870
   Tier 1 matches: 2,472
   Tier 2 matches: 28,485
   Tier 3 matches: 26,405

   Stats coverage: 48.8%
   Memory usage: 40.03 MB


## Next: 03_feature_engineering.ipynb

The clean `matches_base.parquet` is ready for feature engineering:
- Rolling form calculations (last 5/10 games) - **PRE-MATCH features**
- Home/away specific metrics - **PRE-MATCH features**
- Head-to-head history - **PRE-MATCH features**
- Pre-match standings context

### Important: Pre-match vs Post-match Fields
- **Pre-match fields**: Can be used in filter conditions (form, standings, H2H)
- **Post-match fields**: Only for ground truth evaluation (scores, actual stats)